In [ ]:
%pip install pyarrow
%pip install fastparquet


In [1]:
import pandas as pd
import misdirection_analysis
import pyarrow
import fastparquet


In [6]:
week1 = pd.read_csv('data/tracking_week_1.csv')
tracking_df = pd.DataFrame()
gameIds = week1['gameId'].unique()

In [7]:
games = pd.read_csv('data/games.csv')
player_plays = pd.read_csv('data/player_play.csv')

players = pd.read_csv('data/players.csv')
wrs = players[players['position'] == 'WR']

wr_plays = player_plays[player_plays['nflId'].isin(wrs['nflId'])]

plays = pd.read_csv('data/plays.csv')
pass_plays = plays[plays['isDropback'] == True]
pass_plays = pd.merge(pass_plays, wr_plays[['gameId', 'playId', 'premotion_classification']], on=('gameId', 'playId'))

plays = plays[plays['gameId'].isin(gameIds)]
pass_plays = pass_plays[pass_plays['gameId'].isin(gameIds)]
wr_plays = wr_plays[wr_plays['gameId'].isin(gameIds)]

In [8]:


for game in gameIds:
    tracking_df = pd.concat([tracking_df, misdirection_analysis.load_tracking_adjusted(game)], ignore_index=True)


In [27]:
def get_play_separation(gameId, playId, nflId):
    play = tracking_df[(tracking_df['gameId'] == gameId) & (tracking_df['playId'] == playId) & (tracking_df['nflId'] == nflId)]

    try:
        pass_frame = play[play['event'] == 'pass_forward']
        pass_frame = pass_frame['frameId'].values[0]

        before_pass = play[play['frameId'] == pass_frame - 10]
        before_pass = before_pass['separation'].values[0]
        pass_frame = play[play['frameId'] == pass_frame]
        pass_frame = pass_frame['separation'].values[0]

        return pass_frame - before_pass
    except:
        return None



In [28]:
separations = []

for _, play in pass_plays.iterrows():
    player_play = wr_plays[(wr_plays['gameId'] == play['gameId']) & (wr_plays['playId'] == play['playId'])]

    target = player_play[player_play['wasTargettedReceiver'] == 1]
    target = target['nflId']

    if len(target) > 0:
        separations.append(get_play_separation(play['gameId'], play['playId'], target.values[0]))
    else:
        separations.append(None)


In [29]:
pass_plays['separation'] = separations

In [31]:
pass_plays.dropna(subset=['separation'], inplace=True)

anova by motion type

In [35]:
%pip install statsmodels

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 58.5 MB/s eta 0:00:000:00:01
Note: you may need to restart the kernel to use updated packages.


In [36]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

In [37]:
pass_plays.head()

,gameId,playId,playDescription,quarter,down,yardsToGo,possessionTeam,defensiveTeam,yardlineSide,yardlineNumber,...,visitorTeamWinProbilityAdded,expectedPointsAdded,isDropback,pff_runConceptPrimary,pff_runConceptSecondary,pff_runPassOption,pff_passCoverage,pff_manZone,premotion_classification,separation
12,2022091104,3662,(12:51) (Shotgun) J.Hurts pass incomplete shor...,4,3,12,PHI,DET,PHI,35,...,-0.012361,-1.161621,True,NaN,NaN,0,Cover-0,Man,NaN,0.014232
13,2022091104,3662,(12:51) (Shotgun) J.Hurts pass incomplete shor...,4,3,12,PHI,DET,PHI,35,...,-0.012361,-1.161621,True,NaN,NaN,0,Cover-0,Man,NaN,0.014232
14,2022091104,3662,(12:51) (Shotgun) J.Hurts pass incomplete shor...,4,3,12,PHI,DET,PHI,35,...,-0.012361,-1.161621,True,NaN,NaN,0,Cover-0,Man,NaN,0.014232
21,2022091112,1674,(:35) (Shotgun) A.Rodgers pass deep left inten...,2,1,10,GB,MIN,GB,25,...,-0.003262,-1.071627,True,NaN,NaN,0,Cover-2,Zone,NaN,0.299196
22,2022091112,1674,(:35) (Shotgun) A.Rodgers pass deep left inten...,2,1,10,GB,MIN,GB,25,...,-0.003262,-1.071627,True,NaN,NaN,0,Cover-2,Zone,NaN,0.299196


In [38]:
model = ols('separation ~ premotion_classification', data=pass_plays).fit()
anova_table = anova_lm(model)

# Print the ANOVA table
print(anova_table)

                             df      sum_sq   mean_sq         F  PR(>F)
premotion_classification   19.0   53.151863  2.797466  0.524151  0.9483
Residual                  148.0  789.895999  5.337135       NaN     NaN


In [43]:
import numpy as np
pass_plays['motion'] = np.where(pass_plays['premotion_classification'].isna(), 0, 1)

In [44]:
from statsmodels.stats.weightstats import ttest_ind
motion = pass_plays[pass_plays['motion'] == 1]['separation']
no_motion = pass_plays[pass_plays['motion'] == 0]['separation']

t_stat, p_value, df = ttest_ind(motion, no_motion, usevar='unequal')

# Print results
print(f"T-statistic: {t_stat}")
print(f"P-value: {p_value}")
print(f"Degrees of Freedom: {df}")

T-statistic: -0.33023344682459643
P-value: 0.7415534923735824
Degrees of Freedom: 209.54905125339334
